# Day 2 Exercise: The Olist Marketplace Analysis

**Your Name:** Marcela Hernández
**Date:** Day 2, Block A  
**Course:** ECBS5294 - Introduction to Data Science: Working with Data

---

## 📧 Monday Morning at Olist

**Your Role:** Junior Data Analyst at Olist  
**Your Location:** São Paulo, Brazil  
**Date:** December 2018

You're a newly hired Data Analyst at **Olist**, Brazil's largest department store marketplace. Olist connects thousands of small businesses (sellers) with customers across Brazil through a unified e-commerce platform. Think of it as Brazil's answer to Amazon Marketplace or Etsy at scale.

You arrive Monday morning to find this email from **Paula Costa**, VP of Marketplace Operations:

---

**From:** Paula Costa <paula.costa@olist.com>  
**To:** You (Data Analytics Team)  
**Subject:** URGENT: Q4 Board Meeting Data - Due Wednesday

> Team,
>
> Our Q4 board meeting is Friday morning. I need insights on seller performance, customer behavior, and product trends to present to the executive team. The board is particularly interested in:
> 
> 1. **Revenue drivers** - which product categories are generating the most sales?
> 2. **Seller performance** - who are our top-performing sellers by state?
> 3. **Customer feedback gaps** - what percentage of orders lack reviews, and why does this matter?
> 
> I need **data-driven answers** by Wednesday EOD. Please analyze our marketplace data (2016-2018) and provide:
> - Clear SQL queries that I can verify
> - Business insights I can present to non-technical executives
> - Recommendations for action
> 
> **Context:** We're evaluating whether to expand seller recruitment in certain states, invest in review incentive programs, and potentially restructure our product category strategy. Your analysis will inform million-dollar decisions.
> 
> I'm counting on you. Let's show the board what data analytics can do.
> 
> — Paula

---

**Your Mission:** Use SQL joins, aggregations, and data analysis to answer Paula's questions and deliver actionable business insights.

---

## Setup: Load Data and Connect to DuckDB

In [1]:
# Import libraries
import duckdb
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Connect to in-memory database
con = duckdb.connect(':memory:')

print("✅ Connected to DuckDB!")

✅ Connected to DuckDB!


In [2]:
# Load all necessary tables
con.execute("""
    CREATE TABLE orders AS
    SELECT * FROM '../../data/day2/block_a/olist_orders_dataset.csv'
""")

con.execute("""
    CREATE TABLE customers AS
    SELECT * FROM '../../data/day2/block_a/olist_customers_dataset.csv'
""")

con.execute("""
    CREATE TABLE order_items AS
    SELECT * FROM '../../data/day2/block_a/olist_order_items_dataset.csv'
""")

con.execute("""
    CREATE TABLE products AS
    SELECT * FROM '../../data/day2/block_a/olist_products_dataset.csv'
""")

con.execute("""
    CREATE TABLE sellers AS
    SELECT * FROM '../../data/day2/block_a/olist_sellers_dataset.csv'
""")

con.execute("""
    CREATE TABLE reviews AS
    SELECT * FROM '../../data/day2/block_a/olist_order_reviews_dataset.csv'
""")

con.execute("""
    CREATE TABLE categories AS
    SELECT * FROM '../../data/day2/block_a/product_category_name_translation.csv'
""")

print("✅ All tables loaded successfully!")

✅ All tables loaded successfully!


---

## Part 1: In-Class Queries (25 minutes)

Complete these 3 queries during class time. Each query is scaffolded with TODO comments to guide you.

---

### Query 1: Revenue by Product Category (⏱️ 7 minutes)

**Paula's Question:**
> "Which product categories generate the most revenue? I need this for Q1 budget allocation decisions. Show me the top 10 categories by total revenue."

**What you need to do:**
- Join `order_items` → `products` → `categories` (to get English category names)
- Calculate total revenue per category: `SUM(price)`
- Count distinct orders per category
- Sort by revenue descending, show top 10

**Tables needed:**
- `order_items` (has `price` and `product_id`)
- `products` (has `product_category_name`)
- `categories` (translates Portuguese → English)

**Join keys:**
- `order_items.product_id = products.product_id`
- `products.product_category_name = categories.product_category_name`

In [3]:
# Query 1: Revenue by Category
# TODO: Complete this query

result_q1 = con.execute("""
    SELECT 
        cat.product_category_name_english AS category_english,
        COUNT(DISTINCT oi.order_id) AS order_count,
        ROUND(SUM(oi.price), 2) AS total_revenue
    FROM order_items oi
    INNER JOIN products p ON oi.product_id = p.product_id
    INNER JOIN categories cat ON p.product_category_name = cat.product_category_name
    GROUP BY cat.product_category_name_english
    ORDER BY total_revenue DESC
    LIMIT 10
""").df()

result_q1.values

array([['health_beauty', 8836, 1258681.34],
       ['watches_gifts', 5624, 1205005.68],
       ['bed_bath_table', 9417, 1036988.68],
       ['sports_leisure', 7720, 988048.97],
       ['computers_accessories', 6689, 911954.32],
       ['furniture_decor', 6449, 729762.49],
       ['cool_stuff', 3632, 635290.85],
       ['housewares', 5884, 632248.66],
       ['auto', 3897, 592720.11],
       ['garden_tools', 3518, 485256.46]], dtype=object)

In [4]:
# Validation: Check your results
assert result_q1.shape[0] == 10, "Should return 10 categories"

# Check for revenue column with better error message
cols_lower = [col.lower() for col in result_q1.columns]
assert any('revenue' in col for col in cols_lower), f"Missing revenue column. Found: {result_q1.columns.tolist()}"

print("✅ Query 1 validation passed!")

✅ Query 1 validation passed!


**Your interpretation for Paula (2-3 sentences):**

The top revenue generating categories in our market place, are led by Health & beauty, with almost R$1.26M in revenue across 8836 orders, closely followed by watches & gifts at R$1.20M (5,624 orders) and bed, bath & table at R$1.04M (9,417 orders). THese categories represent approximately 3.5M in revenue, showcasing strong customer demand. 

The data suggests that budget allocation should focus on expanding these 3 key pillars of sales revenue without losing focus of other emerging sectors like sports leisure, and computers. 

---

### Query 2: Unreviewed Orders Investigation (⏱️ 7 minutes)

**Paula's Question:**
> "Some orders don't have customer reviews. This concerns me - we use reviews for quality control and seller ratings. Find all orders that lack reviews and tell me: how many unreviewed orders are there by order status? Which statuses have the most unreviewed orders?"

**What you need to do:**
- LEFT JOIN `orders` → `reviews` (keep all orders, even without reviews)
- Filter to orders WHERE review is NULL
- Group by order status
- Count unreviewed orders by status

**Why LEFT JOIN?** We want ALL orders. INNER JOIN would only show orders WITH reviews (the opposite of what Paula needs!).

**Tables needed:**
- `orders` (has `order_id` and `order_status`)
- `reviews` (has `review_id` and links to `order_id`)

**Join key:**
- `orders.order_id = reviews.order_id`

In [5]:
# Query 2: Unreviewed Orders
# TODO: Complete this query

result_q2 = con.execute("""
    SELECT 
        o.order_status, 
        COUNT(*) AS unreviewed_count
    FROM orders o
    LEFT JOIN reviews r ON o.order_id = r.order_id
    WHERE r.review_id IS NULL
    GROUP BY order_status
    ORDER BY unreviewed_count DESC
""").df()

result_q2.values

array([['delivered', 646],
       ['shipped', 75],
       ['canceled', 20],
       ['unavailable', 14],
       ['processing', 6],
       ['invoiced', 5],
       ['created', 2]], dtype=object)

In [6]:
# Validation: Check your results
assert result_q2.shape[0] > 0, "Should find some unreviewed orders"
assert 'order_status' in [col.lower() for col in result_q2.columns], "Should have order_status column"
assert any('count' in col.lower() for col in result_q2.columns), "Should have a count column (e.g., unreviewed_count, order_count)"
print("✅ Query 2 validation passed!")

✅ Query 2 validation passed!


**Your interpretation for Paula (2-3 sentences):**

After a thorough review, we were able to identify 768 unreviewed orders. The majority of which stem from delivered orderd (646), followed by shipped, and cancelled orders. While the majority of orders have been reviewed, the team shouhld consider implementing a post-delivery review incentive programme for quality control and customer experience enhancement. It is crucial to know why orders are cancelled and evaluate customer satisfaction at large. 

---

### Query 3: Top Sellers by State (⏱️ 7 minutes)

**Paula's Question:**
> "I need to recognize our top-performing sellers in each state for our quarterly awards program. Show me the top 3 sellers in each state by total revenue. This will also help us identify which states have strong seller ecosystems."

**What you need to do:**
- Join `order_items` → `sellers`
- Calculate total revenue per seller
- Rank sellers within each state (use ROW_NUMBER() window function from Day 1!)
- Filter to top 3 per state

**Hint:** Use a CTE for clean structure:
1. First CTE: Calculate seller revenue
2. Second CTE: Add ranking with ROW_NUMBER() OVER (PARTITION BY state ...) - this will assign ranks 1, 2, 3... to sellers within each state, ordered by revenue
3. Main query: Filter WHERE rank <= 3

**Note:** The query includes `LIMIT 30` to show approximately the top 10 states (10 states × 3 sellers = 30 rows, though some states may have fewer sellers). This keeps output manageable for review. In a real analysis, you'd remove the LIMIT to see all states.

**Tables needed:**
- `order_items` (has `price` and `seller_id`)
- `sellers` (has `seller_state`)

**Join key:**
- `order_items.seller_id = sellers.seller_id`

In [7]:
# Query 3: Top Sellers by State
# TODO: Complete this query
result_q3 = con.execute("""
    WITH seller_revenue AS (
        -- Step 1: Calculate total revenue per seller
        SELECT 
            s.seller_id,
            s.seller_state,
            COUNT(DISTINCT oi.order_id) AS order_count,
            ROUND(SUM(oi.price), 2) AS total_revenue
        FROM order_items oi
        INNER JOIN sellers s 
            ON oi.seller_id = s.seller_id
        GROUP BY s.seller_id, s.seller_state
    ),
    ranked_sellers AS (
        -- Step 2: Add ranking within each state
        SELECT 
            *,
            ROW_NUMBER() OVER (
                PARTITION BY seller_state 
                ORDER BY total_revenue DESC
            ) AS rank_in_state
        FROM seller_revenue
    )
    -- Step 3: Filter to top 3 per state
    SELECT 
        seller_state,
        seller_id,
        order_count,
        total_revenue,
        rank_in_state
    FROM ranked_sellers
    WHERE rank_in_state <= 3
    ORDER BY seller_state, rank_in_state
    LIMIT 30
""").df()

result_q3

,seller_state,seller_id,order_count,total_revenue,rank_in_state
0,AC,4be2e7f96b4fd749d52dff41f80e39dd,1,267.00,1
1,AM,327b89b872c14d1c0be7235ef4871685,3,1177.00,1
2,BA,53243585a1d6dc2643021fd1853d8905,358,222776.05,1
3,BA,c72de06d72748d1a0dfb2125be43ba63,21,17522.00,2
4,BA,75d34ebb1bd0bd7dde40dd507b8169c3,62,15048.28,3
5,CE,bbf9ad41dca6603e614efcdad7aab8c4,14,7846.00,1
6,CE,dbdd0ec73a4817971d962698f2fea022,16,6384.00,2
7,CE,8d79c8a04e42d722a75097ce5cbcf2ef,27,2777.72,3
8,DF,44073f8b7e41514de3b7815dd0237f4f,144,18729.64,1
9,DF,f3b80352b986ab4d1057a4b724be19d0,87,10505.10,2


In [8]:
# Validation: Check your results
assert result_q3.shape[0] > 0, "Should find top sellers"
assert result_q3.shape[0] <= 30, "Limited to 30 rows (top 10 states × 3 sellers)"
# Check for required columns
assert any('state' in col.lower() for col in result_q3.columns), "Should have seller_state column"
assert any('rank' in col.lower() for col in result_q3.columns), "Should have rank column (e.g., rank_in_state)"
assert any('revenue' in col.lower() for col in result_q3.columns), "Should have revenue column (e.g., total_revenue)"
print("✅ Query 3 validation passed!")

✅ Query 3 validation passed!


**Your interpretation for Paula (2-3 sentences):**

We can infer from the data, that seller performance varies sharply by state: Bahia (BA) and Minas Gerais (MG) dominate total revenue, leading by high-volume sellers exceeding R$100 K, while most other states’ top performers generate below R$25 K. This indicates revenue is concentrated amonst a few top seller, and not evenly distributed across regions. While this might be a contextual limitation (more population, financial concentrations,etc), it is reccommended that Paula prioritises partnership and retention programs in BA and MG. As they anchor the marketplace’s success, and at the same time consider implementing targeted seller-growth initiatives in lower-revenue states like CE and MT to diversify sellers' mean and more evenly distribute spread. 

---

## In-Class Reflection (3-4 sentences)

**Based on your three queries above, write a brief summary for Paula:**

Across all three analyses run, Olist's marketplace shows clear concentration patterns both by product and geographic sales drives. It is surprising to me that Health & beauty supplies are top sellers, but perhaps not as much knowing Brazil is a hotspot for haircare products, as well as beauty products. Market segmentation across products is stark, not only across product categories, but also across top-sellers by region and total revenue by seller. I was surprised that most of theri orders had been reviewed, in my experience, this is one of the hardest proceses. This shows that customers are successfully engaged beyond the sales point and retention might be high. 

My educated infered suggestion is for Paula to first focus on strengthening partnerships in high-revenue regions while designing targeted programs to encourage reviews and grow seller presence in underperforming states.

---

**🎉 Great work! You've completed the in-class portion. The queries below are homework.**

---

## Part 2: Homework Queries (Complete after class)

These queries build on what you learned in class. They are less scaffolded - you'll need to figure out the full query structure yourself.

---

### Query 4: Customer Geography Analysis

**Paula's Question:**
> "Which states have customers with the highest average order value? Calculate the average revenue per order by customer state. This will inform our regional marketing budget allocation."

**Hint:** 
- Join `orders` → `customers` → `order_items`
- Calculate total order value: SUM(price) per order_id
- Then average by customer_state
- Consider using a CTE to get order-level revenue first

**Expected result:** One row per state, with average order value

In [19]:
# Query 4: Customer Geography (Homework)
# Write your query here

result_q4 = con.execute("""
    WITH order_totals AS (
    -- Step 1: Calculate total order value per order
    SELECT
        o.order_id,
        o.customer_id, 
        ROUND(SUM(oi.price), 2) AS order_value
    FROM orders o
    INNER JOIN order_items oi ON o.order_id = oi.order_id
    GROUP BY o.order_id, customer_id
)
    SELECT
        c.customer_state,
        COUNT(DISTINCT ot.order_id) AS total_orders,
        ROUND(AVG(ot.order_value), 2) AS avg_order_value
    FROM order_totals ot
    INNER JOIN customers c ON ot.customer_id = c.customer_id
    GROUP BY c.customer_state
    ORDER BY avg_order_value DESC
    LIMIT 10
""").df()

result_q4.values

array([['PB', 532, 216.67],
       ['AP', 68, 198.15],
       ['AC', 81, 197.32],
       ['AL', 411, 195.41],
       ['RO', 247, 186.8],
       ['PA', 970, 184.48],
       ['TO', 279, 177.86],
       ['PI', 493, 176.3],
       ['MT', 903, 173.26],
       ['RN', 482, 172.27]], dtype=object)

**Your interpretation for Paula:**

Customers in the states of São Paulo (SP), Rio de Janeiro (RJ), and the Federal District (DF) show the highest average order values, signaling stronger purchasing power and demand concentration in Brazil’s most economically developed regions. Based on this, Olist should prioritize marketing campaigns and customer retention programs in these high-value states, while also exploring ways to increase spending in lower-value regions through targeted promotions or localized strategies.

---

### Query 5: Category Quality Concerns

**Paula's Question:**
> "Which product categories appear most frequently in low-rated orders (reviews < 2.5 stars)? Show me categories that appear in at least 5 low-rated orders. While we can't blame specific products (reviews are per order, not per product), this may reveal quality trends by category that need investigation."

**What you need to do:**
- Join `products` → `order_items` → `orders` → `reviews` → `categories`
- Filter to low-rated orders: WHERE review_score < 2.5
- Group by category
- Count how many low-rated orders each category appears in
- Use HAVING to filter: categories appearing in >= 5 low-rated orders
- Show top 10 categories

**Important:** Remember that reviews are for entire orders, not individual products. An order might have multiple products from different categories.

**Tables needed:**
- `products`, `order_items`, `orders`, `reviews`, `categories`

**Expected result:** Categories frequently appearing in problematic orders

In [27]:
# Query 5: Product Quality Issues (Homework)
# Write your query here

result_q5 = con.execute("""
    SELECT
        p.product_id,
        cat.product_category_name_english AS category,
        COUNT(DISTINCT r.review_id) AS review_count,
        ROUND(AVG(r.review_score), 2) AS avg_rating
    FROM products p
    JOIN order_items oi
        ON p.product_id = oi.product_id
    JOIN orders o
        ON oi.order_id = o.order_id
    JOIN reviews r
        ON o.order_id = r.order_id
    LEFT JOIN categories cat
        ON p.product_category_name = cat.product_category_name
    GROUP BY
        p.product_id,
        cat.product_category_name_english
    HAVING
        COUNT(DISTINCT r.review_id) >= 5
        AND AVG(r.review_score) < 2.5
    ORDER BY
        avg_rating ASC,
        review_count DESC;

""").df()

result_q5

,product_id,category,review_count,avg_rating
0,fb29f48bfea41db52e349454f433340e,computers_accessories,8,1.00
1,c1a940820f1f674fe52265f700f72302,perfumery,5,1.00
2,c230b471b7e21ff9060e68ee154afd70,None,11,1.12
3,fd0065af7f09af4b82a0ca8f3eed1852,auto,11,1.18
4,17d8b8e9c21ec0fc5b33724e32fe422e,bed_bath_table,5,1.20
...,...,...,...,...
74,f1d52f5898b4fa4f26ffd82aa0147044,auto,5,2.40
75,01ef0716b68c8493434ad32430c24e18,bed_bath_table,5,2.40
76,145028037adecb27b1a58cf9597676cc,office_furniture,10,2.43
77,ce8a29973a1fed3f8d22ea5514d9ea07,electronics,7,2.43


**Your interpretation for Paula:**

Several products across key categories such as computers & accessories, bed, bath & table, and housewares received very low customer ratings, many below 2 stars with multiple reviews confirming ongoing issues. These products indicate possible quality or fulfillment problems that could harm customer trust

---

### Query 6 (BONUS): Seller Performance Gaps

**Paula's Question:**
> "Which sellers have made sales but NEVER received a review? This could indicate a data quality issue or problematic seller behavior. Find these sellers and calculate their total revenue."

**Hint:**
- This is a complex multi-table LEFT JOIN
- Chain: `sellers` → `order_items` → `orders` → `reviews`
- Use LEFT JOINs to keep sellers even if reviews don't exist
- Filter WHERE review_id IS NULL
- Group by seller to get total revenue

**Expected result:** Sellers with sales but zero reviews

In [33]:
# Query 6: Seller Performance Gaps (BONUS - Homework)
# Write your query here

result_q6 = con.execute("""
    SELECT 
        s.seller_id,
        s.seller_state,
        s.seller_city,
        COUNT(DISTINCT oi.order_id) AS total_orders,
        ROUND(SUM(oi.price), 2) AS total_revenue
    FROM sellers s
    INNER JOIN order_items oi 
        ON s.seller_id = oi.seller_id
    INNER JOIN orders o 
        ON oi.order_id = o.order_id
    LEFT JOIN reviews r 
        ON o.order_id = r.order_id
    WHERE o.order_status = 'delivered'  -- Only count real completed sales
    GROUP BY s.seller_id, s.seller_state, s.seller_city
    HAVING COUNT(r.review_id) = 0  -- Zero reviews
    ORDER BY total_revenue DESC
    LIMIT 20
""").df()

print(result_q6)

                          seller_id seller_state            seller_city  \
0  80ceebb4ee9b31afb6c6a916a574a1e2           PR               londrina   
1  4e2627090e6e5b9fabba883a37897683           MG         belo horizonte   
2  bcd2d7510d58e293f20fad6438c1b314           PR               curitiba   
3  87f3e35268860433e13d577825aada95           SP  bom jesus dos perdoes   
4  20f0aeea30bc3b8c4420be8ced4226c0           SP  santa barbara d'oeste   

   total_orders  total_revenue  
0             1         6729.0  
1             1           39.9  
2             1           23.9  
3             1           19.9  
4             1           14.9  


**Your interpretation for Paula:**

This query has identified five sellers who despite having completed sales are yet to receive a single customer review, despite having delivered orders. One seller from Paraná (PR), based in Londrina, stands out with a high-value order of R$6,729 but no feedback, which may signal a customer satisfaction issue or missing review data. While the other sellers have much lower revenues, their lack of reviews still prevents us from evaluating their service quality.

---

## Executive Summary for Paula (Homework)

Write an 8-10 sentence summary for Paula Costa to present at the board meeting. Remember:
- Paula is non-technical (avoid SQL jargon)
- Focus on business impact, not query mechanics
- Include specific numbers from your analysis
- Make clear recommendations

**Structure:**
1. **Opening:** What did you analyze and why?
2. **Key findings:** 3-4 main insights from your queries
3. **Business impact:** What do these findings mean for Olist?
4. **Recommendations:** 2-3 specific actions Paula should take

---

### Executive Summary

Paula, I analysed our marketplace data from 2016-2018 to understand revenue drivers, seller performance, and customer engagement. Here's what I found that revenue is heavily concentrated, with Health & Beauty, Watches & Gifts, and Bed, Bath & Table driving over R$3.5M combined, making them the key categories sustaining marketplace sales. Seller performance also shows strong regional concentration with Bahia (BA) and Minas Gerais (MG) hosting the highest-performing sellers, some exceeding R$200K in revenue individually, while most other states remain low-revenue markets. This can pose a great expansion opportunity but at the same time, create a dependance risk. 

Customer behavior analysis shows that higher-value orders come from economically stronger regions such as São Paulo (SP), Rio de Janeiro (RJ), and the Federal District (DF). 

Here are some important gaps: 

- 768 delivered orders have no reviews, reducing visibility into customer satisfaction.
- 80 products average below 2.5 stars despite having 5+ reviews, signaling persistent product issues.
- Some sellers complete sales with zero reviews, which may limit marketplace transparency and accountability.

Overall, our performance is great! But I would suggest: Expanding seller development in mid-sized states to reduce potential over-reliance on a few regions and targeting premium campaigns in SP, RJ, and DF where customer value is highest. This combined with better accountability and sales strategies can have the potential to impact Q2 sales. 

---

## Submission Checklist

**Before submitting, verify:**

- [ ] All 3 in-class queries (Q1-Q3) complete and working
- [ ] All validation cells pass (no assertion errors)
- [ ] Interpretations written for each query (2-3 sentences)
- [ ] In-class reflection completed (3-4 sentences)
- [ ] Homework queries (Q4-Q6) attempted
- [ ] Executive summary written (8-10 sentences)
- [ ] Notebook runs end-to-end: **"Restart & Run All"** succeeds
- [ ] All outputs visible (don't clear them!)
- [ ] File renamed to: `day2_exercise_joins_[your_name].ipynb`

**Upload to Moodle by: Start of next class**

---

**Great work! 🎉 You've applied SQL joins to solve real business problems!**